Q1. Deterministic noise depends on $\mathcal{H}$, as some models approximate $f$ better than others. Assume $\mathcal{H}' \subset \mathcal{H}$ and that $f$ is fixed. In general (but not necessarily in all cases), if we use $\mathcal{H}'$ instead of $\mathcal{H}$, how does deterministic noise behave?

Since $\mathcal{H}$' is a subset of $\mathcal{H}$, it's at most as complex as $\mathcal{H}'$, and since deterministic noise is generally a relation between hypothesis complexity and target complexity, and $f$ is fixed, deterministic noise should increase.  

__[b] In general, deterministic noise will increase. __

In the following problems use the data provided in the files

    http://work.caltech.edu/data/in.dta
    http://work.caltech.edu/data/out.dta

as a training and test set respectively. Each line of the files corresponds to a two dimensional input $\mathbf{x} = (\mathbf{x}_1, \mathbf{x}_2)$, so that $\mathcal{X} = \mathbb{R}^2$, followed by the corresponding label from $\mathcal{Y} = \{−1, 1\}$. We are going to apply Linear Regression with a non-linear transformation for classification. The nonlinear transformation is given by

$$\phi(x_1, x_2) = (1, x_1, x_2, x_1^2, x_2^2, x_1x_2, |x_1 - x_2|, |x_1 + x_2|)$$

Recall that the classification error is defined as the fraction of misclassified points. 

In [156]:
import numpy as np
import pandas as pd
import requests

# pandas won't import the stata file directly (?), so using numpy instead

# request data, format into numpy array
train_req = requests.get("http://work.caltech.edu/data/in.dta")
train = np.array([row.split() for row in train_req.text.split("\r\n")[:-1]]).astype(float)

test_req = requests.get("http://work.caltech.edu/data/out.dta")
test = np.array([row.split() for row in test_req.text.split("\r\n")[:-1]]).astype(float)


def transform(data):
    ''' applies specified nonlinear transform on data by adding columns'''
    trans = addFeature(data, np.square(data[:, 1]))
    trans = addFeature(trans, np.square(data[:, 2]))
    trans = addFeature(trans, data[:, 1]*data[:, 2])
    trans = addFeature(trans, np.abs(data[:, 1] - data[:, 2]))
    trans = addFeature(trans, np.abs(data[:, 1] + data[:, 2]))
    
    
def addFeature(data, feature):
    ''' helper function for adding feature '''
    return np.concatenate((data, feature.reshape(data.shape[0], 1)), axis = 1)
    


In [245]:
class LinearRegression():
    ''' Linear Regression Learning Model '''
    def __init__(self, train, test):
        ''' assuming data is formatted so that last row are ys, rest is xs'''
        self.x = train[:, :-1]
        
        # add column of 1s to data
        self.x = np.concatenate((np.ones(train.shape[0]).reshape(train.shape[0], 1), self.x), axis =1)
        
        # seperate out ys
        self.y = train[:, -1]
        
        # apply non-linear transform
        self.x = self.transform(self.x)
        
        # initialize weights to 0
        self.w = np.zeros(self.x.shape[1])
        
        # seperate and apply transformation to testing data
        self.test_x = test[:, :-1]
        self.test_x = np.concatenate((np.ones(test.shape[0]).reshape(test.shape[0], 1), self.test_x), axis =1)

        self.test_y = test[:, -1]        
        self.test_x = self.transform(self.test_x)
    
    def regress(self, l = 0):
        ''' solve for regularized weights with lambda = l; defaults to no regularization '''
        self.w = np.linalg.lstsq(np.dot(self.x.T, self.x) + l * np.identity(self.x.shape[1]), np.dot(self.x.T, self.y))[0]
        
        
    def classify(self, data):
        ''' hard threshold regression for classification '''
        return np.where(np.inner(self.w, data) > 0, 1.0, -1.0)
    
    def classificationError(self):
        ''' calculate in and out of sample error '''
        ein = self.y[self.classify(self.x) != self.y].shape[0]
        eout = self.test_y[self.classify(self.test_x) != self.test_y].shape[0]
        
        # return fraction of misaclassified points in and out of sample
        return (ein / self.y.shape[0], eout / self.test_y.shape[0])
        
        
    def transform(self, data):
        ''' applies specified nonlinear transform on data by adding columns'''
        trans = self.addFeature(data, np.square(data[:, 1]))
        trans = self.addFeature(trans, np.square(data[:, 2]))
        trans = self.addFeature(trans, data[:, 1]*data[:, 2])
        trans = self.addFeature(trans, np.abs(data[:, 1] - data[:, 2]))
        trans = self.addFeature(trans, np.abs(data[:, 1] + data[:, 2]))

        return trans

    def addFeature(self, data, feature):
        ''' helper function for adding feature '''
        return np.concatenate((data, feature.reshape(data.shape[0], 1)), axis = 1)
    

In [283]:
t = LinearRegression(train, test)
t.regress()
t.classificationError()


(0.02857142857142857, 0.084)

Q2. Run Linear Regression on the training set after performing the non-linear transformation.
What values are closest (in Euclidean distance) to the in-sample and
out-of-sample classification errors, respectively?

__ A2. [a] 0.03, 0.08 __


Q3. Now add weight decay to Linear Regression, that is, add the term $\frac{\lambda}{N} \sum_{i=0}^7 w_i^2 $ to the squared in-sample error, using $\lambda = 10^k$. What are the closest values to the in-sample and out-of-sample classification errors, respectively, for $k = −3$?
Recall that the solution for Linear Regression with Weight Decay was derived in class.

In [247]:
t.regress(l = 10**-3)
t.classificationError()

(0.02857142857142857, 0.08)

__ A3. [d] 0.03, 0.08 __

Q4. Now, use $k = 3$. What are the closest values to the new in-sample and out-of-sample classification errors, respectively?

In [248]:
t.regress(l = 10**3)
t.classificationError()


(0.37142857142857144, 0.436)

__ A4. [e] 0.4, 0.4 __

Q5. What value of $k$, among the following choices, achieves the smallest out-of-sample classification error?

In [284]:
minimum = float('inf')
min_k = 4
for k in range(-2, 3):
    l = 10**k
    t.regress(l)
    if t.classificationError()[1] < minimum:
        minimum = t.classificationError()[1]
        min_k = k
        
print("Minimum k is {}, with out-of-sample error of {}".format(min_k, minimum))

Minimum k is -1, with out-of-sample error of 0.056


__ A5. [d] -1 __

Q6. What value is closest to the minimum out-of-sample classification error achieved by varying $k$ (limiting $k$ to integer values)?

In [287]:
minimum = float('inf')
min_k = 4
for k in range(-10, 10):
    l = 10**k
    t.regress(l)
    if t.classificationError()[1] < minimum:
        minimum = t.classificationError()[1]
        min_k = k
print("Minimum k is {}, with out-of-sample error of {}".format(min_k, minimum))

Minimum k is -1, with out-of-sample error of 0.056


__ A6.  [b] 0.06 __

Polynomial models can be viewed as linear models in a space $\mathcal{Z}$, under a nonlinear transform $\Phi: \mathcal{X} \rightarrow \mathcal{Z}$. Here, $\Phi$ transforms the scalar $x$ into a vector $\mathbf{z}$ of Legengre polynomials, $z = (1, L_1(x), L_2(x), \ldots, L_Q (x))$. Our hypothesis set will be expresed as a linear combination of these polynomials, 

$$\mathcal{H}_Q = \left \{ h \ | \ h(x) = \mathbf{w}^T \mathbf{z} = \sum_{q=0}^{Q} w_q L_q(x) \right \},$$

where $L_0(x) = 1$. 

Q7. Consider the following hypothesis set defined by the constraint: 

$$\mathcal{H}(Q, C, Q_0) = \{ h \ |  \ h(x) = \mathbf{w}^T \mathbf{z} \in \mathcal{H}_Q; w_q = C \text{ for } q \geq Q_0 \}, $$

which of the following statements is correct: 

[a] $\mathcal{H}(10, 0, 3) \cup \mathcal{H}(10, 0, 4) = \mathcal{H}_3$

The first constrained set is effectively just $\mathcal{H}_2$, since all weights greater than or equal to 3 are set to 0, similarly the second set is just $\mathcal{H}_3$ so their union is $\mathcal{H}_3$. 

[b] $\mathcal{H}(10, 1, 3) \cup \mathcal{H}(10, 1, 4) = \mathcal{H} (10, 1, 3)$

The first sets all weights greater than order 3 to 1. The second does the same to 4, and is thus a subset of the first. 

[c] $\mathcal{H}(10, 0, 3) \cap \mathcal{H} (10, 0, 4) = \mathcal{H}_2$

The first gets rid of all terms greater than order 3; the second gets rid of all terms greater than order 4. What remains after their intersection is thus all terms less than 3, or $\mathcal{H}_2$. 

[d] $\mathcal{H}(10, 1, 3) \cap \mathcal{H}(10, 1, 4) = \mathcal{H}(10, 1, 4)$

As with [b], the second term is a subset of the first, so their intersection results in that subset. 

__A7. [c] $\mathcal{H}(10, 0, 3) \cap \mathcal{H}(10, 0, 4) = \mathcal{H}_2$ __

Q8. A fully connected Neural Network has $L = 2$; $d^{(0)} = 5$, $d^{(1)} = 3$, $d^{(2)} = 1$. If only products of the form $w_{ij}^{(l)} x_i^{l-1}$, $w_{ij}^{(l)} \delta _j^{(l)}$, and $x_i^{(l-1)} \delta_j^{(l)}$ count as operations (even for $x_0^{(l-1)} = 1$), without counting anything else, which of the following is the closest to the number of operations in a single iteration of backpropagation (using SGD on one data point)?

A8. Backpropagation has three components: a feedforward step where you compute all $x_j^{(l)}$, a backpropagate step where you compute all $\delta_j^{(l)}$, and an update step where you need to update every weight, $w_{ij}^{(l)}$. The feedforward requires you to compute every $x_j^{(l)}$, except obviously for constant values so we have $6 \cdot 3 = 18$ for going forward from the input to the hidden layer, and $4 \cdot 1 = 4$ operations for going from the hidden layer to the output (for a total of 22 operations). For the backpropogation step, we also don't need to compute the deltas for the constant values, but here as well for the input values (since they have no signal feeding them; they are features). Then, given the above criteria for what counts as an operation, since 

$$\delta^{(l-1)} = (1 - (x_i^{(l-1)})^2) \sum_{j=1}^{d^{(l)}} w_{ij}^{(l)} \delta_j^{(l)}$$

while, for the final layer (and using a squared error without loss of generality), we have, 

$$\delta_j^{(1)} = \frac{\partial \mathbf{e}(w)}{\partial s_j^{(1)}} =  \frac{\partial}{\partial s_j^{(1)}} (\theta (s_j^{(L)} - y_n) )^2 = 2(x_j^{1} - y_n) \theta'(s) $$

it's clear that we only need to count operations on the hidden layers, of which there are only 3 non-constant ones. 

Finally, for the update step, we have to update everything but constant values, i.e., the same as the feedforward step. 

Thus, for all three, we need $22 + 3 + 22 = 47$. 

__ A8. [d] 45 __

Let us call every ‘node’ in a Neural Network a unit, whether that unit is an input variable or a neuron in one of the layers. Consider a Neural Network that has $10$ input units (the constant $x_0^{(0)}$ is counted here as a unit), one output unit, and $36$ hidden units (each $x_0^{(l)}$
is also counted as a unit). The hidden units can be arranged in any number of layers $l = 1, \ldots, L−1$, and each layer is fully connected to the layer above it.

Q9. What is the minimum possible number of weights that such a network can have? 

A9. Minimizing the number of weights, when each layer is fully connected to the layer above it, would require each layer to consist of just a single neuron (and a necessary constant unit), since more than one neuron per layer would result in more than one weight per neuron/input in order to be fully connected. Thus we have the 10 input variables feeding into one neuron for 10 weights, 18 neurons feeding into each other with 17 constants for 34 more weights, then the last one and its weight feeding into the output neuron for 2 more weights. The total number of weights is then 46. 

__ A9. [a] 46 __


Q10. What is the maximum possible number of weights that such a network can have? 

If we assume that there are only 2 layers, we can formulate the problem into an optimization problem with a constraint, and solve it using Lagrange multipliers. If the number of hidden layers is 2, let $f(x,y)$ be the function which counts the number of weights given the number of neurons in the first and second layer respectively, and $g(x,y) = x + y - 36$ be the constraint (that there are only 36 hidden  units). 

Then, 

\begin{align*}
&f(x,y) = 10(x-1) + x(y-1) + y = 9x + xy + y - 10 \\
&g(x,y) = x + y - 36 \\
\end{align*}

So that the Lagrangian is, 
$$\mathcal{L}(x,y,\lambda) = 9x + xy + y - 10 - \lambda(x + y - 36) = 9x + xy - 10 - \lambda x - \lambda y + \lambda 36 $$

with optimal solution given by setting the gradient to the zero vector, 

$$\nabla \mathcal{L} (x, y, \lambda) = (9 + y - \lambda, x - \lambda + 1, x+ y-36) = \mathbf{0}$$

Giving us a system, 

$$ \left [ \begin{array}{ccc}
0 & 1 & -1 \\
1 & 0 & -1 \\
1 & 1 & 0 
\end{array} \right ] \left [ \begin{array} {c}
x \\
y \\
\lambda 
\end{array} \right ] = \left [ \begin{array}{c} 
-9 \\
-1 \\
36 \end{array} \right ] $$

In [277]:
A = np.array([[0, 1, -1], [1, 0, -1], [1, 1, 0]])
ans = np.dot(np.linalg.inv(A), np.array([-9, -1, 36]))
print(9*ans[0] + ans[0]*ans[1]+14-10)

510.0


To verify that 510 is really the optimal solution, we could see that it's the largest available option, or repeat the process with more layers (so for $3$, note that $f(x,y,z) = 10(x-1) + x(y-1) + y(z-1) + z$ and $g(x,y,z) = x + y + z - 36$, etc.) and observe (not shown) that the number continues to go down as layers increase. 

__ A10. [e] 510 __